In [131]:
import math
import re
from collections import defaultdict

## Part 3

> Preprocessing Train/Test Files

In [132]:
def preprocess(tfile):
    with open(tfile, 'r') as file:
        lst = file.read()
    pattern = r'[^\w\s]'
    text = re.sub(pattern, '', lst)
    text = text.lower()
    data = text.strip().split()
    return data

> Train the Model using Train Data

In [133]:
def train_model(train_data):
    unigrams = defaultdict(float)
    bigrams = defaultdict(float)
    
    for word in train_data:
        unigrams[word] = unigrams.get(word, 0) + 1

    for i in range(len(train_data) - 1):
        bigram = (train_data[i], train_data[i + 1])
        bigrams[bigram] = bigrams.get(bigram, 0) + 1
        
    return unigrams, bigrams

> Calculate Probabilities of Unigrams and Bigrams

In [134]:
def cal_probabilities(unigrams, bigrams):
    unigram_prob = defaultdict(float)
    total_words = sum(unigrams.values())   
    bigram_prob = defaultdict(float)

    for word in unigrams:
        unigram_prob[word] = unigrams.get(word, 0) / total_words
        
        
    for bigram in bigrams:
        prev_word,current_word = bigram
        bigram_prob[bigram] = bigrams.get(bigram,0) / unigrams.get(prev_word, 0)

    
    return unigram_prob, bigram_prob

In [135]:
training_file = "/Users/chaitradaddolu/Desktop/nlp/Assignment1-CXD210026-KXS210121/A1_DATASET/train.txt"
test_file = "/Users/chaitradaddolu/Desktop/nlp/Assignment1-CXD210026-KXS210121/A1_DATASET/val.txt"
train_data = preprocess(training_file)
test_data = preprocess(test_file)
N = len(test_data)
n = len(train_data)
print(N)
print(n)

8699
79399


In [136]:
unigrams, bigrams = train_model(train_data)
vocab_size = len(unigrams)

In [137]:
#print(unigrams)

In [138]:
#print(bigrams)

In [139]:
unigram_probablities_of_train_data, bigram_probablities_of_train_data = cal_probabilities(unigrams, bigrams)

In [140]:
# for word in unigram_probablities_of_train_data:
#     print(word, "-", unigram_probablities_of_train_data[word])

In [141]:
# for bigram in bigram_probablities_of_train_data:
#     print(bigram, "-", bigram_probablities_of_train_data[bigram])

## Part 4

>  Calculate Unigram total log probability using UNK method by creating Implicit vocabulary with size v and choosing top v words by frequency and replacing rest with UNK. Here size is choosen as 3076 where the bottom tokens have occurence less than frequency of 2.


In [142]:
def calculate_unigram_log_prob_unk_method1(test_data, unigrams):
    total_log_prob = 0
    total_words = sum(unigrams.values())
    unigram_probs = defaultdict(float)
    
    unigrams_sorted = sorted(unigrams, key=unigrams.get, reverse = True)
    
    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0)) / total_words

    unigram_probs['<UNK>'] = (len(unigrams)-3076) / total_words
    

    for word in test_data:
        if word not in unigram_probs:
            word = '<UNK>'
        total_log_prob += math.log(unigram_probs[word])


    return total_log_prob 

> Calculate Unigram total log probability using UNK method as described above and do laplace smoothing on the same.


In [143]:
def calculate_unigram_log_prob_unk_method1_laplace(test_data, unigrams):
    total_log_prob = 0
    total_words = sum(unigrams.values())
    unigram_probs = defaultdict(float)
    vocab_size_new = 3076
    
    unigrams_sorted = sorted(unigrams, key = unigrams.get, reverse = True)

    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0) + 1) / (total_words + vocab_size_new)

    unigram_probs['<UNK>'] = ((len(unigrams) - vocab_size_new) + 1) / (total_words + vocab_size_new)
    
    for word in test_data:
        if word not in unigram_probs:
            word = '<UNK>'
        total_log_prob += math.log(unigram_probs[word])
        
    return total_log_prob

> Calculate Unigram total log probability using UNK method as described above and do add-k smoothing on the same.

In [144]:
def calculate_unigram_log_prob_unk_method1_add_k(test_data, unigrams, k):
    total_log_prob = 0
    total_words = sum(unigrams.values())
    unigram_probs = defaultdict(float)
    vocab_size_new = 3076
     
    unigrams_sorted = sorted(unigrams, key = unigrams.get, reverse=True)

    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0) + k) / (total_words + (k * vocab_size_new))

    unigram_probs['<UNK>'] = ((len(unigrams)-vocab_size_new) + k) / (total_words + ( k * vocab_size_new))
    
    for word in test_data:
        if word not in unigram_probs:
            word = '<UNK>'
        total_log_prob += math.log(unigram_probs[word])
        
    return total_log_prob

> Calculate Unigram total log probability using just laplace or add-k smoothing .

In [145]:
def calculate_unigram_log_prob_with_smoothing(test_data, unigrams, smoothing_method, vocab_size, k):
    total_log_prob = 0

    for word in test_data:
        if smoothing_method == 'laplace':
            prob = (unigrams.get(word, 0) + 1) / (sum(unigrams.values()) + vocab_size)
        elif smoothing_method == 'add-k':
            prob = (unigrams.get(word, 0) + k) / (sum(unigrams.values()) + k * vocab_size)
            
        total_log_prob += math.log(prob)

    return total_log_prob

> Calculate Unigram total log probability using UNK token tagging where we add a UNK tag in unigrams from train dataset where the probabilities are recalculated using laplace smoothing.

In [146]:
def calculate_unigram_log_prob_unk_method2_with_laplace(test_data, unigrams, vocab_size):
    total_log_prob = 0
    total_words = sum(unigrams.values())
    unigram_probs = defaultdict(float)
    
    for word in unigrams:
        unigram_probs[word] = (unigrams.get(word, 0)+ 1) / (total_words + vocab_size)
        
    unigram_probs['<UNK>'] = 1 / (total_words + vocab_size)


    for word in test_data:
        if word not in unigrams:
            word = '<UNK>'
        total_log_prob += math.log(unigram_probs[word])
    
    return total_log_prob

> Calculate Unigram total log probability using UNK token tagging where we add a UNK tag in unigrams from train dataset where the probabilities are recalculated using add-k smoothing.

In [147]:
def calculate_unigram_log_prob_unk_method2_add_k(test_data, unigrams, vocab_size, k):
    total_log_prob = 0
    total_words = sum(unigrams.values())
    total_words = sum(unigrams.values())
    unigram_probs = defaultdict(float)
    
    for word in unigrams:
        unigram_probs[word] = (unigrams.get(word, 0) + k) / (total_words + k * vocab_size)
        
    unigram_probs['<UNK>'] = ( k )/(total_words +  k * vocab_size)


    for word in test_data:
        if word not in unigrams:
            word = '<UNK>'
        total_log_prob += math.log(unigram_probs[word])

    return total_log_prob

> Calculate Bigram total log probability using UNK token tagging method by creating Implicit vocabulary with size v and choosing top v words by frequency and replacing rest with UNK. Here size is choosen as 3076 where the bottom tokens have occurence less than frequency of 2.

In [148]:
def calculate_bigram_log_prob_unk_method1(test_data, bigrams, unigrams):
    total_log_prob = 0
    N = len(test_data)
    unigram_probs = defaultdict(float)
    bigram_probs = defaultdict(float)
    bigrams_new = defaultdict(float)
    vocab_size_new = 3076
    
    unigrams_sorted = sorted(unigrams, key = unigrams.get, reverse = True)
    
    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0)) + 1
        
    unigram_probs['<UNK>'] = (len(unigrams) - vocab_size_new)
    
    for bigram in bigrams:
        prev_word,current_word = bigram
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
        bigram = (prev_word, current_word)
        
        bigrams_new[bigram] = bigrams_new.get(bigram, 0) + 1

        
    for i in range(1, N):
        prev_word = test_data[i - 1]
        current_word = test_data[i]
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
        
        bigram = (prev_word, current_word)
        
        if bigram not in bigrams_new:
            bigram = ('<UNK>', '<UNK>')
        bigram_probs[bigram] = bigrams_new.get(bigram, 0) / unigram_probs[prev_word]
        total_log_prob += math.log(bigram_probs[bigram])
        
    return total_log_prob

> Using the same above method calculated bigrams total log probability of test_data with laplace smoothing.

In [149]:
def calculate_bigram_log_prob_unk_method1_laplace(test_data, bigrams, unigrams):
    total_log_prob = 0
    N = len(test_data)
    unigram_probs = defaultdict(float)
    bigram_probs = defaultdict(float)
    bigrams_new = defaultdict(float)
    vocab_size_new = 3076
    
    unigrams_sorted = sorted(unigrams, key = unigrams.get, reverse = True)
    
    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0)) + 1

        
    unigram_probs['<UNK>'] = (len(unigrams) - vocab_size_new)
    
    for bigram in bigrams:
        prev_word, current_word = bigram
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
        bigram = (prev_word, current_word)
        
        bigrams_new[bigram] = bigrams_new.get(bigram, 0) + 1

        
    for i in range(1, N):
        prev_word = test_data[i - 1]
        current_word = test_data[i]
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
            
        bigram = (prev_word, current_word)
        
        if bigram not in bigrams_new:
            bigram = ('<UNK>', '<UNK>')
            
        bigram_probs[bigram] = (bigrams_new.get(bigram, 0) + 1) / (unigram_probs[prev_word] + vocab_size_new)
        total_log_prob += math.log(bigram_probs[bigram])

    return total_log_prob

> Using the same above method calculated bigrams total log probability of test_data with add-k smoothing.

In [150]:
def calculate_bigram_log_prob_unk_method1_add_k(test_data, bigrams, unigrams, k):
    total_log_prob = 0
    N = len(test_data)
    unigram_probs = defaultdict(float)
    bigram_probs = defaultdict(float)
    bigrams_new = defaultdict(float)
    vocab_size_new = 3076
    
    unigrams_sorted = sorted(unigrams, key = unigrams.get, reverse = True)
    
    for i in range(3076):
        word = unigrams_sorted[i]
        unigram_probs[word] = (unigrams.get(word, 0)) + 1

        
    unigram_probs['<UNK>'] = (len(unigrams) - vocab_size_new)

    
    for bigram in bigrams:
        prev_word, current_word = bigram
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
        bigram = (prev_word, current_word)
        
        bigrams_new[bigram] = bigrams_new.get(bigram,0) + 1

        
    for i in range(1, N):
        prev_word = test_data[i - 1]
        current_word = test_data[i]
        if prev_word not in unigram_probs:
            prev_word = '<UNK>'
        if current_word not in unigram_probs:
            current_word = '<UNK>'
        
        bigram = (prev_word, current_word)
        
        if bigram not in bigrams_new:
            bigram = ('<UNK>', '<UNK>')
        bigram_probs[bigram] = (bigrams_new.get(bigram) + k) / (unigram_probs[prev_word] + k * vocab_size_new)
        total_log_prob += math.log(bigram_probs[bigram])

    return total_log_prob

> Calculate total log probabilities using just laplace and add-k smoothing

In [151]:
def calculate_bigram_log_prob_smoothing(test_data, unigrams, bigrams, vocab_size, smoothing_method, k):
    total_log_prob = 0
    N = len(test_data)

    for i in range(1, N):
        prev_word = test_data[i - 1]
        current_word = test_data[i]

        if smoothing_method == 'laplace':
            prob = (bigrams.get((prev_word, current_word), 0) + 1) / (unigrams.get(prev_word, 0) + vocab_size)
        elif smoothing_method == 'add-k':
            prob = (bigrams.get((prev_word, current_word), 0) + k) / (unigrams.get(prev_word, 0) + k * vocab_size)

        total_log_prob += math.log(prob)
        
    return total_log_prob

## Part 5

> To calculate perplexity from total log probability and size of test data

In [152]:
def calculate_perplexity(total_log_prob, N):
    
    return math.exp(-total_log_prob / N)
    

> Calculating total log probabilites by invoking the already defined methods for unigrams

In [153]:
log_prob_unigram_unk_method1 = calculate_unigram_log_prob_unk_method1(test_data, unigrams)
log_prob_unigram_unk_method1_laplace = calculate_unigram_log_prob_unk_method1_laplace(test_data, unigrams)
log_prob_unigram_unk_method1_add_k = calculate_unigram_log_prob_unk_method1_add_k(test_data, unigrams, 0.5)
log_prob_unigram_laplace_smoothing = calculate_unigram_log_prob_with_smoothing(test_data, unigrams, 'laplace', vocab_size, 1)
log_prob_unigram_add_k_smoothing = calculate_unigram_log_prob_with_smoothing(test_data, unigrams, 'add-k', vocab_size, 0.5)
log_prob_unigram_unk_method2_laplace = calculate_unigram_log_prob_unk_method2_with_laplace(test_data, unigrams, vocab_size)
log_prob_unigram_unk_method2_add_k = calculate_unigram_log_prob_unk_method2_add_k(test_data, unigrams, vocab_size, 0.5)

> Calculating perplexity by invoking the already defined perplexity method for unigrams

In [154]:
perplexity_unigram_unk_method1 = calculate_perplexity(log_prob_unigram_unk_method1, N)
perplexity_unigram_unk_method1_laplace = calculate_perplexity(log_prob_unigram_unk_method1_laplace, N)
perplexity_unigram_unk_method1_add_k = calculate_perplexity(log_prob_unigram_unk_method1_add_k, N)
perplexity_unigram_laplace_smoothing = calculate_perplexity(log_prob_unigram_laplace_smoothing, N)
perplexity_unigram_add_k_smoothing = calculate_perplexity(log_prob_unigram_add_k_smoothing, N)
perplexity_unigram_unk_method2_laplace = calculate_perplexity(log_prob_unigram_unk_method2_laplace, N)
perplexity_unigram_unk_method2_add_k = calculate_perplexity(log_prob_unigram_unk_method2_add_k, N)

In [155]:
print(f"Unigram Perplexity with unk method 1: {perplexity_unigram_unk_method1}")
print(f"Unigram Perplexity with unk method 1 and laplace: {perplexity_unigram_unk_method1_laplace}")
print(f"Unigram Perplexity with unk method 1 and add-k: {perplexity_unigram_unk_method1_add_k}")
print(f"Unigram Perplexity with laplace smoothing: {perplexity_unigram_laplace_smoothing}")
print(f"Unigram Perplexity with add-k smoothing: {perplexity_unigram_add_k_smoothing}")
print(f"Unigram Perplexity with unk-laplace method 2: {perplexity_unigram_unk_method2_laplace}")
print(f"Unigram Perplexity with unk-add-k method 2: {perplexity_unigram_unk_method2_add_k}")

Unigram Perplexity with unk method 1: 331.65965712657663
Unigram Perplexity with unk method 1 and laplace: 334.572345391625
Unigram Perplexity with unk method 1 and add-k: 332.9410792379095
Unigram Perplexity with laplace smoothing: 550.4715746437497
Unigram Perplexity with add-k smoothing: 555.2862387883032
Unigram Perplexity with unk-laplace method 2: 550.4715746437497
Unigram Perplexity with unk-add-k method 2: 555.2862387883032


> Calculating total log probabilites by invoking the already defined methods for bigrams

In [156]:
log_prob_bigram_unk_method1 = calculate_bigram_log_prob_unk_method1(test_data, bigrams, unigrams)
log_prob_bigram_unk_method1_laplace = calculate_bigram_log_prob_unk_method1_laplace(test_data, bigrams, unigrams)
log_prob_bigram_unk_method1_add_k = calculate_bigram_log_prob_unk_method1_add_k(test_data, bigrams, unigrams, 0.5)
log_prob_bigram_laplace_smoothing = calculate_bigram_log_prob_smoothing(test_data, unigrams, bigrams, vocab_size, 'laplace', 1)
log_prob_bigram_add_k_smoothing1 = calculate_bigram_log_prob_smoothing(test_data, unigrams, bigrams, vocab_size, 'add-k', 0.5)

> Calculating perplexity by invoking the already defined perplexity method for unigrams

In [157]:
perplexity_bigram_unk_method1 = calculate_perplexity(log_prob_bigram_unk_method1, N)
perplexity_bigram_unk_method1_laplace = calculate_perplexity(log_prob_bigram_unk_method1_laplace, N)
perplexity_bigram_unk_method1_add_k = calculate_perplexity(log_prob_bigram_unk_method1_add_k, N)
perplexity_bigram_laplace_smoothing = calculate_perplexity(log_prob_bigram_laplace_smoothing, N)
perplexity_bigram_add_k_smoothing1 = calculate_perplexity(log_prob_bigram_add_k_smoothing1, N)

In [158]:
print(f"Bigram Perplexity with unk method 1: {perplexity_bigram_unk_method1}")
print(f"Bigram Perplexity with unk method 1 and laplace: {perplexity_bigram_unk_method1_laplace}")
print(f"Bigram Perplexity with unk method 1 and add-k: {perplexity_bigram_unk_method1_add_k}")
print(f"Bigram Perplexity with laplace smoothing: {perplexity_bigram_laplace_smoothing}")
print(f"Bigram Perplexity with add-k smoothing 1: {perplexity_bigram_add_k_smoothing1}")

Bigram Perplexity with unk method 1: 44.64114699183954
Bigram Perplexity with unk method 1 and laplace: 447.9366911925982
Bigram Perplexity with unk method 1 and add-k: 316.6566028361528
Bigram Perplexity with laplace smoothing: 1320.2702645843851
Bigram Perplexity with add-k smoothing 1: 975.3721300180292


## Calculating perplexities on train data for uni and bigrams

In [159]:
log_prob_unigram_unk_method1_train = calculate_unigram_log_prob_unk_method1(train_data, unigrams)
log_prob_unigram_unk_method1_laplace_train = calculate_unigram_log_prob_unk_method1_laplace(train_data, unigrams)
log_prob_unigram_unk_method1_add_k_train = calculate_unigram_log_prob_unk_method1_add_k(train_data, unigrams, 0.5)
log_prob_unigram_laplace_smoothing_train = calculate_unigram_log_prob_with_smoothing(train_data, unigrams, 'laplace', vocab_size, 1)
log_prob_unigram_add_k_smoothing_train = calculate_unigram_log_prob_with_smoothing(train_data, unigrams, 'add-k', vocab_size, 0.5)
log_prob_unigram_unk_method2_laplace_train = calculate_unigram_log_prob_unk_method2_with_laplace(train_data, unigrams, vocab_size)
log_prob_unigram_unk_method2_add_k_train = calculate_unigram_log_prob_unk_method2_add_k(train_data, unigrams, vocab_size, 0.5)

In [160]:
perplexity_unigram_unk_method1_train = calculate_perplexity(log_prob_unigram_unk_method1_train, n)
perplexity_unigram_unk_method1_laplace_train = calculate_perplexity(log_prob_unigram_unk_method1_laplace_train, n)
perplexity_unigram_unk_method1_add_k_train = calculate_perplexity(log_prob_unigram_unk_method1_add_k_train, n)
perplexity_unigram_laplace_smoothing_train = calculate_perplexity(log_prob_unigram_laplace_smoothing_train, n)
perplexity_unigram_add_k_smoothing_train = calculate_perplexity(log_prob_unigram_add_k_smoothing_train, n)
perplexity_unigram_unk_method2_laplace_train = calculate_perplexity(log_prob_unigram_unk_method2_laplace_train, n)
perplexity_unigram_unk_method2_add_k_train = calculate_perplexity(log_prob_unigram_unk_method2_add_k_train, n)

In [161]:
print(f"Unigram Perplexity with unk method 1 train: {perplexity_unigram_unk_method1_train}")
print(f"Unigram Perplexity with unk method 1 and laplace train: {perplexity_unigram_unk_method1_laplace_train}")
print(f"Unigram Perplexity with unk method 1 and add-k train: {perplexity_unigram_unk_method1_add_k_train}")
print(f"Unigram Perplexity with laplace smoothing train: {perplexity_unigram_laplace_smoothing_train}")
print(f"Unigram Perplexity with add-k smoothing train: {perplexity_unigram_add_k_smoothing_train}")
print(f"Unigram Perplexity with unk-laplace train method 2: {perplexity_unigram_unk_method2_laplace_train}")
print(f"Unigram Perplexity with unk-add-k train method 2: {perplexity_unigram_unk_method2_add_k_train}")

Unigram Perplexity with unk method 1 train: 381.20140975076384
Unigram Perplexity with unk method 1 and laplace train: 382.4474713719824
Unigram Perplexity with unk method 1 and add-k train: 381.5502932620644
Unigram Perplexity with laplace smoothing train: 529.447063921777
Unigram Perplexity with add-k smoothing train: 524.6629051608547
Unigram Perplexity with unk-laplace train method 2: 529.447063921777
Unigram Perplexity with unk-add-k train method 2: 524.6629051608547


In [162]:
log_prob_bigram_unk_method1_train = calculate_bigram_log_prob_unk_method1(train_data, bigrams, unigrams)
log_prob_bigram_unk_method1_laplace_train = calculate_bigram_log_prob_unk_method1_laplace(train_data, bigrams, unigrams)
log_prob_bigram_unk_method1_add_k_train = calculate_bigram_log_prob_unk_method1_add_k(train_data, bigrams, unigrams, 0.5)
log_prob_bigram_laplace_smoothing_train = calculate_bigram_log_prob_smoothing(train_data, unigrams, bigrams, vocab_size, 'laplace', 1)
log_prob_bigram_add_k_smoothing_train1 = calculate_bigram_log_prob_smoothing(train_data, unigrams, bigrams, vocab_size, 'add-k', 0.5)


In [163]:
perplexity_bigram_unk_method1_train = calculate_perplexity(log_prob_bigram_unk_method1_train, n)
perplexity_bigram_unk_method1_laplace_train = calculate_perplexity(log_prob_bigram_unk_method1_laplace_train, n)
perplexity_bigram_unk_method1_add_k_train = calculate_perplexity(log_prob_bigram_unk_method1_add_k_train, n)
perplexity_bigram_laplace_smoothing_train = calculate_perplexity(log_prob_bigram_laplace_smoothing_train, n)
perplexity_bigram_add_k_smoothing1_train = calculate_perplexity(log_prob_bigram_add_k_smoothing_train1, n)

In [164]:
print(f"Bigram Perplexity with unk method 1 train: {perplexity_bigram_unk_method1_train}")
print(f"Bigram Perplexity with unk method 1 and laplace train: {perplexity_bigram_unk_method1_laplace_train}")
print(f"Bigram Perplexity with unk method 1 and add-k train: {perplexity_bigram_unk_method1_add_k_train}")
print(f"Bigram Perplexity with laplace smoothing train: {perplexity_bigram_laplace_smoothing_train}")
print(f"Bigram Perplexity with add-k smoothing 1 train: {perplexity_bigram_add_k_smoothing1_train}")


Bigram Perplexity with unk method 1 train: 174.3369949437968
Bigram Perplexity with unk method 1 and laplace train: 1621.2261744384784
Bigram Perplexity with unk method 1 and add-k train: 1233.1386552876177
Bigram Perplexity with laplace smoothing train: 981.1156826539259
Bigram Perplexity with add-k smoothing 1 train: 620.2636955844678
